# Howso Engine &#x2014; JSON/YAML as Features

#### Note - This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using JSON/YAML as features in a Trainee to represent complex data structures in a single feature.

In [1]:
import pandas as pd

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## 1. Load Data

We use a small, toy-dataset to demonstrate this functionality where each case represents a character in a game.

In [2]:
data = pd.DataFrame([
    {"name": "Nerissa", "level": 14, "class": "Fighter", "inventory": "{\"saddle\": 1, \"trident\": 2, \"potion\": 5}"},
    {"name": "Maaya", "level": 13, "class": "Warlock", "inventory": "{\"sword\": 1, \"lute\": 1, \"potion\": 2}"},
    {"name": "Taxuul", "level": 15, "class": "Cleric", "inventory": "{\"hammer\": 1, \"metal bar\": 20, \"potion\": 1}"},
])

data

,name,level,class,inventory
0,Nerissa,14,Fighter,"{""saddle"": 1, ""trident"": 2, ""potion"": 5}"
1,Maaya,13,Warlock,"{""sword"": 1, ""lute"": 1, ""potion"": 2}"
2,Taxuul,15,Cleric,"{""hammer"": 1, ""metal bar"": 20, ""potion"": 1}"


## 2. Infer Feature Attributes

The feature-attributes for the ``inventory`` feature are automatically inferred to be JSON.

In [3]:
features = infer_feature_attributes(data)
features

{'name': {'type': 'nominal', 'original_type': {'data_type': 'string'}},
 'level': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'min': 7.0, 'max': 20.0, 'allow_null': False}},
 'class': {'type': 'nominal', 'original_type': {'data_type': 'string'}},
 'inventory': {'type': 'continuous',
  'data_type': 'json',
  'original_type': {'data_type': 'string'}}}

## 3. Train & Analyze Data

In [4]:
t = Trainee(features=features)
t.train(data)
t.analyze()

Version 16.0.1 of Howso Engine™ is available. You are using version 16.0.0.


## 4. React

### 4.1. Discriminative React

First, we use a discriminative react to determine a likely inventory.

In [5]:
discriminative_action = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
)["action"]

discriminative_action.loc[:, "inventory"]

0    {"saddle":1,"trident":2,"potion":2.64786820840...
Name: inventory, dtype: object

### 4.2. Generative React

We can also use a generative react to generate any number of more or less surprising inventories.

With higher desired conviction, we will see JSON that more closely resembles the training data.

In [6]:
generative_reaction_25 = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
    desired_conviction=25,
    num_cases_to_generate=5,
)["action"]

generative_reaction_25.loc[:, "inventory"]

0    {"metal bar":20,"hammer":1,"potion":1}
1           {"lute":1,"sword":1,"potion":2}
2           {"lute":1,"sword":1,"potion":2}
3       {"saddle":1,"trident":2,"potion":5}
4       {"saddle":1,"trident":2,"potion":5}
Name: inventory, dtype: object

With a lower desired conviction, we will see JSON that looks very different from the training data.

In [7]:
generative_reaction_1 = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
    desired_conviction=1,
    num_cases_to_generate=5,
)["action"]

generative_reaction_1.loc[:, "inventory"]

0    {"metal bar":12.649569258253415,"hammer":1,"po...
1    {"metal bar":20,"hammer":1.9426061856191794,"p...
2    {"metal bar":-2,"hammer":0,"potion":-1.4890958...
3    {"metal bar":{},"hammer":"OhcJqC","potion":"Oh...
4                {"metal bar":0,"hammer":1,"potion":1}
Name: inventory, dtype: object

In [8]:
t.delete()